In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# Set the path to your dataset Train folder
dataset_path = r"D:\vs jupyter\cloudconselage\college\drive-download-20230809T132623Z-001\Potato\Train"

# Set the parameters for data preprocessing and model training
image_size = (224, 224)
batch_size = 32
epochs = 20
num_heads = 4
dropout_rate = 0.2

# Load the dataset
def load_dataset(path):
    images = []
    labels = []
    classes = os.listdir(path)
    num_classes = len(classes)

    for i, class_name in enumerate(classes):
        class_path = os.path.join(path, class_name)
        image_names = os.listdir(class_path)
        for image_name in image_names:
            image_path = os.path.join(class_path, image_name)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, image_size)
                images.append(image)
                labels.append(i)

    images = np.array(images, dtype='float32')
    labels = np.array(labels, dtype='int32')

    return images, labels, num_classes

# Split the dataset into training and testing sets
def split_dataset(images, labels, test_size=0.2):
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42)
    return x_train, x_test, y_train, y_test

# Load and preprocess the dataset
images, labels, num_classes = load_dataset(dataset_path)
x_train, x_test, y_train, y_test = split_dataset(images, labels)

# Preprocess input images using EfficientNetB0 preprocessing
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=Input(shape=(image_size[0], image_size[1], 3)))

# Freeze the base model
base_model.trainable = False

# Create the transformer model
inputs = Input(shape=(image_size[0], image_size[1], 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(dropout_rate)(x)
x = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs, x)

# Compile the model
loss_fn = SparseCategoricalCrossentropy()
optimizer = Adam()
accuracy_metric = SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss*100)
print("Test Accuracy:", accuracy*100)


Epoch 1/20
23/23 [==============================] - 27s 1s/step - loss: 0.7397 - sparse_categorical_accuracy: 0.7000 - val_loss: 0.4573 - val_sparse_categorical_accuracy: 0.8556
Epoch 2/20
23/23 [==============================] - 33s 1s/step - loss: 0.3657 - sparse_categorical_accuracy: 0.9153 - val_loss: 0.3023 - val_sparse_categorical_accuracy: 0.9111
Epoch 3/20
23/23 [==============================] - 34s 1s/step - loss: 0.2437 - sparse_categorical_accuracy: 0.9458 - val_loss: 0.2320 - val_sparse_categorical_accuracy: 0.9444
Epoch 4/20
23/23 [==============================] - 34s 1s/step - loss: 0.1876 - sparse_categorical_accuracy: 0.9611 - val_loss: 0.1980 - val_sparse_categorical_accuracy: 0.9556
Epoch 5/20
23/23 [==============================] - 33s 1s/step - loss: 0.1559 - sparse_categorical_accuracy: 0.9764 - val_loss: 0.1666 - val_sparse_categorical_accuracy: 0.9611
Epoch 6/20
23/23 [==============================] - 33s 1s/step - loss: 0.1313 - sparse_categorical_accuracy: 